In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [2]:
import pickle
import pandas as pd

import os

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
# Load February data
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

Question 1

In [7]:
# mean predicted duration
print(y_pred.mean())

16.191691679979066


Question 2

In [10]:
year = 2021
month = 2

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
pd.Series(y_pred, name="predictions")

0         14.539865
1         13.740422
2         15.593339
3         15.188118
4         13.817206
            ...    
990108    12.433246
990109    12.910885
990110    15.834923
990111    16.783176
990112    19.654626
Name: predictions, Length: 990113, dtype: float64

In [17]:
df_result = pd.merge(df["ride_id"], pd.Series(y_pred, name="predictions"), 
right_index=True, left_index=True)

df_result.to_parquet(
    "result.parquet",
    engine="pyarrow",
    compression=None,
    index=False
)

In [18]:
df_result

,ride_id,predictions
1,2021/02_1,13.740422
2,2021/02_2,15.593339
3,2021/02_3,15.188118
4,2021/02_4,13.817206
5,2021/02_5,15.757813
...,...,...
990108,2021/02_990108,12.433246
990109,2021/02_990109,12.910885
990110,2021/02_990110,15.834923
990111,2021/02_990111,16.783176


In [20]:
# get file size
os.path.getsize("result.parquet")/1000000

18.761728